# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from TrainCovid19Infections import clean_data
from sklearn.externals import joblib
import json
import os
import numpy as np
import pandas as pd
import shutil

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## Initialize Workspace

In [2]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
# choose a name for experiment
experiment_name = 'Covid19VaccinationExperiment'
experiment=Experiment(ws, experiment_name)
experiment

Workspace name: OptimizePipeline
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingMLFunctionnalities


Name,Workspace,Report Page,Docs Page
Covid19VaccinationExperiment,OptimizePipeline,Link to Azure Machine Learning studio,Link to Documentation


## Create Cluster

Get cluster if it exists else create one

In [3]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-15T15:14:10.800000+00:00', 'errors': None, 'creationTime': '2021-03-10T15:50:24.915795+00:00', 'modifiedTime': '2021-03-10T15:50:40.833531+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

I Chose a COVID-19 World Vaccination Dataset that holds a track of the world vaccination including the name of the country, Which vaccines have been used by country, and how many have been vaccinated by Country.

Since the covid-19 vaccination is among the hottest subjects in the world, and as a member of the society being interested in such statistic calculations can help further scientists or even regular people to better understand the global effect of this vaccine all over the world.

I used Kaggle's API to download the Dataset.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True)
    
    
    #ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,7.507100e+04,75071.000000,75071.000000,7.507100e+04,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,...,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000,75071.000000
mean,6.485801e+05,5093.479253,5039.755345,1.675972e+04,114.016558,113.020131,7663.025592,65.660352,64.702308,157.272631,...,27.742569,7.896042,5.033854,17442.886039,236.446233,7.259702,23.529798,2.553074,69.490341,0.668392
std,4.591167e+06,32050.216562,31571.899202,1.059525e+05,673.121748,651.876939,15267.263219,167.271383,140.214663,309.017189,...,12.393051,6.485994,4.373118,19620.435364,133.508033,4.287317,33.345754,2.520139,17.548525,0.245824
min,0.000000e+00,-74347.000000,-6223.000000,0.000000e+00,-1918.000000,-232.143000,0.000000,-2153.437000,-276.825000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.680000e+02,1.000000,4.429000,1.000000e+01,0.000000,0.000000,121.875000,0.062000,0.740000,1.580000,...,19.600000,3.008000,1.783000,2921.909000,140.448000,4.610000,0.000000,0.700000,66.470000,0.557000
50%,7.683000e+03,55.000000,62.429000,1.280000e+02,1.000000,0.857000,1021.996000,5.803000,7.479000,19.284000,...,29.000000,5.440000,3.212000,10849.297000,233.070000,6.930000,0.000000,2.000000,74.080000,0.737000
75%,8.532200e+04,656.000000,660.357500,1.666500e+03,11.000000,11.429000,6956.768500,55.036500,61.244000,133.959000,...,38.000000,13.260000,8.353000,25129.341000,318.949000,9.750000,47.782000,3.600000,78.490000,0.828000
max,1.202181e+08,880902.000000,739564.429000,2.660456e+06,17895.000000,14424.000000,146107.552000,8652.658000,2648.773000,2268.843000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.957000


In [5]:
# preview the first 10 rows of the dataset
df.head(10)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
5,AFG,Asia,Afghanistan,2020-02-29,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
6,AFG,Asia,Afghanistan,2020-03-01,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
7,AFG,Asia,Afghanistan,2020-03-02,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
8,AFG,Asia,Afghanistan,2020-03-03,2.0,1.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
9,AFG,Asia,Afghanistan,2020-03-04,4.0,2.0,0.429,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511


In [6]:
# Use the clean_data function to clean your data.
x, y = clean_data(df)
data = pd.concat([x,y],axis=1)
data.head()

,date,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,new_cases
0,737479,1.0,0.0,0.0,0.0,0.0,0.026,0.026,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,737480,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,737481,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,737482,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,737483,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [7]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.3,random_state = 42,shuffle=True)

In [8]:
# Create necessary folders
if "automl_training" not in os.listdir():
    os.mkdir("./automl_training")
if "data" not in os.listdir():
    os.mkdir("./data")

# store training_dataset into it using datastore
script_folder = './automl_training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', script_folder)
project_folder = './pipeline-project'

## AutoML Configuration
TODO: Explain why you chose the automl settings and cofiguration you used below.
The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions 

In [9]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('training/training_data.csv')
#Create an experiment for the AutoML testing script
exp = Experiment(workspace=ws, name="Covid19AutoMlExperiment")

# Get the dataset from the data folder
datastore.upload_files(files = ['training/training_data.csv'],
                       target_path ='./data/',
                       overwrite = True,
                       show_progress = True)
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('./data/training_data.csv'))])
#training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 2.0,
    "max_concurrent_iterations": 3,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = cpu_cluster,
                             training_data = training_dataset,
                             label_column_name = 'new_cases',
                             featurization= 'auto',
                             path=project_folder,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [10]:
# Experiment Submission
tag = {"Covid19Infections": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

Running on remote.
No run_configuration provided, running on Covid19Cluster with default configuration
Running on remote compute: Covid19Cluster
Parent Run ID: AutoML_355924c4-4287-4ff0-8cf5-b6e52cc68add

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:      

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_355924c4-4287-4ff0-8cf5-b6e52cc68add',
 'target': 'Covid19Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-16T09:52:56.869038Z',
 'endTimeUtc': '2021-03-16T11:54:33.25044Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'Covid19Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Covid19VaccinationExperiment","subscription_id":"c04b3d3f-4994-454d-96ff-aa3f2050b57f","resource_group":"testingMLFunctionnalities","workspace_name":"OptimizePipeline","region":"eastus2","compute_target":"Covid19Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":n

## Best Model

In [12]:
# Retrieve and save best model.
best_automl_run, model = remote_run.get_output()

# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()
print(model._final_estimator)

# Show the Accuracy of that run
print('Best accuracy: {}'.format(best_run_metrics['accuracy']))

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                               max_

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# register the best model
best_model = best_automl_run.register_model(model_name = "automl_covid19_model", model_path = './outputs/')
print(best_model.name, best_model.id, best_model.version, sep='\t')


NameError: name 'best_automl_run' is not defined

## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
with open('score.py') as f:
    print(f.read())

{
    "entryScript": "score.py",
    "sourceDirectory": "./working_dir"
}


In [18]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

service_name = 'Covid19-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)
validate_configuration()
service.wait_for_deployment(show_output=True)
print(service.state)
print_deploy_configuration()

NameError: name 'inference_config' is not defined

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

delete()